### Importing libraries

In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
from decimal import Decimal
import scipy.ndimage
import matplotlib.pyplot as plt

In [2]:
file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
# file_path2 = 'TestData/amblyopiaStudy MetricsSanaTest.tsv'

dataExport = pd.read_excel(file_path1)
# dataMetric = pd.read_csv(file_path2, sep='\t')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)


In [3]:
# display(dataExport.columns[0:50])
# print(dataExport['Recording resolution height'])
# print(dataExport['Recording resolution width'])
# print(dataExport['Viewport position X'])
# print(dataExport['Viewport width'])
# print(dataExport['Original Media height'])

Checking to find the end of calibration index

In [49]:
from sklearn.impute import KNNImputer

def preprocessing(dataExport):
    imputer = KNNImputer(n_neighbors=2)

    #dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name']
    calibrationEnd = dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name'].index.to_list()[-1]
    calibrationEnd+=1
    dataExport = dataExport[["Eyetracker timestamp [μs]", "Event", "Event value", "Gaze point X [MCS norm]", "Gaze point Y [MCS norm]", "Eye movement type", 
    "Presented Stimulus name", 'AOI hit [slothStimulus-final (1) - sloth1]',
       'AOI hit [slothStimulus-final (1) - sloth2]',
       'AOI hit [slothStimulus-final (1) - sloth3]',
       'AOI hit [slothStimulus-final (1) - sloth4]',
       'AOI hit [slothStimulus-final (1) - sloth5]',
       'AOI hit [slothStimulus-final (1) - sloth6]',
       'AOI hit [slothStimulus-final (1) - sloth7]',
       'AOI hit [slothStimulus-final (1) - sloth8]',
       'AOI hit [slothStimulus-final (1) - sloth9]',
       'AOI hit [slothStimulus-final (1) - sloth10]',
       'AOI hit [slothStimulus-final (1) - sloth11]',
       'AOI hit [catStimulus-final (1) - cat1]',
        'AOI hit [catStimulus-final (1) - cat2]',
       'AOI hit [catStimulus-final (1) - cat3]',
       'AOI hit [catStimulus-final (1) - cat4]',
       'AOI hit [catStimulus-final (1) - cat5]',
       'AOI hit [catStimulus-final (1) - cat6]',
       'AOI hit [catStimulus-final (1) - cat7]',
       'AOI hit [catStimulus-final (1) - cat8]',
       'AOI hit [catStimulus-final (1) - cat9]',
       'AOI hit [catStimulus-final (1) - cat10]',
       'AOI hit [catStimulus-final (1) - cat11]',
        'Gaze point left X [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 
                         'Gaze point right Y [MCS norm]']]



    #Only isolate non calibration data
    dataExportWithoutCalibration = dataExport[calibrationEnd:]

    #Drop all columns with eye movement type of eyes not found
    dataExportWithoutCalibration = dataExportWithoutCalibration[dataExportWithoutCalibration['Eye movement type'] != 'EyesNotFound']

    #Convert the x and y coordinates into pixels and then into inches using ppi of 92.
    
    #Check this -
    #Normalized --> absolute pixels on the screen - ()
    dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']]*20.82 

    dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']]*11.71 

    #Find the first timestamp that isn't nan: 
    # nonNanTimeStampSlothStart = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]
    nonNanTimeStampSlothStart = dataExportWithoutCalibration.loc[dataExportWithoutCalibration['Eyetracker timestamp [μs]'].first_valid_index(), 'Eyetracker timestamp [μs]']

    #normalizes TIME when the calibration ends and stimulus sloth starts (at 0)
    dataExportWithoutCalibration['Eyetracker timestamp [μs]'] = dataExportWithoutCalibration['Eyetracker timestamp [μs]'] - nonNanTimeStampSlothStart #dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]

    #normalizes TIME when the sloth ends and cats start (at 0)
    #find the index where the cat stimulus begins
    indices = dataExportWithoutCalibration.index[dataExportWithoutCalibration['Presented Stimulus name'] == "catStimulus-final (1)"]

    #Find the beginning time for normalization
    beginTime = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].loc[indices[0]]
    index_of_catBegin = (dataExportWithoutCalibration.index.tolist()).index(indices[0])
    onylCat = dataExportWithoutCalibration.index[index_of_catBegin:]
    #iterate through all times for cat stimulus and update
    for i in onylCat:
        dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] = dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] - beginTime

    display(dataExportWithoutCalibration.head())
    display(dataExportWithoutCalibration.tail())

    #interpolate data
    columns_to_interpolate = ['Gaze point X [MCS norm]', 'Gaze point Y [MCS norm]', 'Gaze point left X [MCS norm]', 
                          'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 'Gaze point right Y [MCS norm]']
    
    

    # Use the subset parameter to interpolate only the specified columns

    dataExportWithoutCalibration[columns_to_interpolate] = dataExportWithoutCalibration[columns_to_interpolate].interpolate(method='polynomial', order =2 )
#     dataExportWithoutCalibration['Gaze point right X [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])
#     dataExportWithoutCalibration['Gaze point right Y [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])


    nan_df = dataExportWithoutCalibration.isna().sum()  # or df.isnull()
    
    display(dataExportWithoutCalibration['Gaze point right X [MCS norm]'][0:100])
    #isolate only saccade data
    mask = dataExportWithoutCalibration["Eye movement type"] == "Saccade"
    dfSaccade = dataExportWithoutCalibration[mask]
    return dfSaccade, dataExportWithoutCalibration



In [34]:
def primaryAndSecondarySaccade(dfSaccade, times_dict): 
    AOI_list = ['AOI hit [slothStimulus-final (1) - sloth1]',
           'AOI hit [slothStimulus-final (1) - sloth2]',
           'AOI hit [slothStimulus-final (1) - sloth3]',
           'AOI hit [slothStimulus-final (1) - sloth4]',
           'AOI hit [slothStimulus-final (1) - sloth5]',
           'AOI hit [slothStimulus-final (1) - sloth6]',
           'AOI hit [slothStimulus-final (1) - sloth7]',
           'AOI hit [slothStimulus-final (1) - sloth8]',
           'AOI hit [slothStimulus-final (1) - sloth9]',
           'AOI hit [slothStimulus-final (1) - sloth10]',
           'AOI hit [slothStimulus-final (1) - sloth11]',
           'AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']

    aoi_cat =  ['AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']




    primarySaccadeIndex = defaultdict(int)
    primarySaccadeTime = defaultdict(int)
    dfSaccadeReset = dfSaccade.reset_index()
    index_of_catBegin = dfSaccadeReset[dfSaccadeReset['Presented Stimulus name'] == "catStimulus-final (1)"].iloc[0, 0]
    

    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            time = row['Eyetracker timestamp [μs]']
            if i in aoi_cat:
                if index > index_of_catBegin: 
                    if time/1000000 > (times_dict[i] + 0.2) and (time/1000000 < (times_dict[i] + 2.5)): 
                        primarySaccadeIndex[i] = index
                        primarySaccadeTime[i] = time/1000000
                        break


            else: 
                if time/1000000 > (times_dict[i] + 0.2) and (time/1000000 < (times_dict[i] + 2.5)): 
                    primarySaccadeIndex[i] = index
                    primarySaccadeTime[i] = time/1000000
                    break
    primarySaccadeTime
   


    ## TIME + AOI SECONDARY SACCADES
    secondarySaccadesTimeAndAOI = defaultdict(int)
    for i in times_dict.keys():
        all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]
        for index, row in all_possible_hits.iterrows(): 
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (0.250 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTimeAndAOI[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (0.250 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTimeAndAOI[i] = index
                    break
    display(secondarySaccadesTimeAndAOI)

    secondarySaccadesTime = defaultdict(int)
    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (0.250 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTime[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (0.250 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTime[i] = index
                    break

    return primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI


In [35]:
# dataExport[calibrationEnd:]
# dataExport['Eyetracker timestamp [μs]'].isna().sum()
# dataExport['Eyetracker timestamp [μs]'].count()
# dfSaccade['Eyetracker timestamp [μs]'].isna().sum()

### Identify the primary saccade 

In [36]:
# ppi = math.sqrt(1080**2 + 1920**2) / 24
# print(ppi)
# viewing_dist = 22*ppi
# print(viewing_dist)

#convert the normalizaed values into pixels and then into 
#we have l is 22 inches


#### Finding amplitude for both eyes, right, eye, left eye, for primary saccades. 

In [37]:
def amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in primarySaccadeIndex.items():

        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = Decimal(str(row1['Gaze point X [MCS norm]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [MCS norm]'].iloc[0]))

            #convert the index label to actual index
            index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == value].index

            index_of_desired_row = index_of_desired_row -1
            new_value = df_reset_orginal.loc[index_of_desired_row, 'index']

            row2 = df_reset_orginal[df_reset_orginal['index'] == int(new_value)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [MCS norm]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [MCS norm]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (360/np.pi) * np.tan(distance/(2*22))

            x1 = row1['Gaze point left X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point left Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [MCS norm]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeLeftEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))
            
            print(key,value)
            

            x1 = row1['Gaze point right X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point right Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [MCS norm]'].iloc[0]
            print(x1,y1,x2,y2)
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))


   
    
    print("Primary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Primary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Primary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")

    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Finding amplitude for both eyes, right eye, left eye for secondary saccades 



### Secondary saccades (only time constraint)

In [38]:
def secondarySaccadeTimeAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTime): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTime.items():

        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = Decimal(str(row1['Gaze point X [MCS norm]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [MCS norm]'].iloc[0]))

            #convert the index label to actual index
            index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == value].index

            index_of_desired_row = index_of_desired_row -1
            new_value = df_reset_orginal.loc[index_of_desired_row, 'index']

            row2 = df_reset_orginal[df_reset_orginal['index'] == int(new_value)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [MCS norm]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [MCS norm]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (360/np.pi) * np.tan(distance/(2*22))

            x1 = row1['Gaze point left X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point left Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [MCS norm]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeLeftEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))

            x1 = row1['Gaze point right X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point right Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [MCS norm]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))


    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
   
    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Secondary saccades (time and AOI constraint)

In [39]:
def secondarySaccadeTimeandAOIAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTimeAndAOI):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTimeAndAOI.items():

        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = Decimal(str(row1['Gaze point X [MCS norm]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [MCS norm]'].iloc[0]))
            #convert the index label to actual index 

            index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == value].index

            index_of_desired_row = index_of_desired_row -1
            new_value = df_reset_orginal.loc[index_of_desired_row, 'index']

            row2 = df_reset_orginal[df_reset_orginal['index'] == int(new_value)]

            x2 = Decimal(str(row2.loc[:,'Gaze point X [MCS norm]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [MCS norm]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (360/np.pi) * np.tan(distance/(2*22))

            x1 = row1['Gaze point left X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point left Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [MCS norm]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeLeftEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))

            x1 = row1['Gaze point right X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point right Y [MCS norm]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [MCS norm]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [MCS norm]'].iloc[0]
            
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (360/np.pi) * np.tan(distance/(2*22))

    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    
   
    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Finding Saccade Latency - using given timestamps

In [40]:
def saccadeLatency(primarySaccadeTime, times_dict):
    saccadeLatency =  defaultdict(int)
    i = 0
    for key,value in primarySaccadeTime.items(): 
        saccadeLatency[key] = abs(times_dict[key] - value)
    
    print("Primary Saccade Latency (Mean and STD)")
    values = list(saccadeLatency.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return saccadeLatency



### Accuracy between primary saccade and stimuli: 

In [41]:
def accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    #}

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in primarySaccadeIndex.items():
        if np.isnan(value)==False:
            
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance
    
    print("Distance between primary saccade & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")

    return accuracy_dist
    

### Accuracy between secondary saccade (time constraint) and stimuli: 

In [42]:
def accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTime.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance


    
    print("Distance between secondary saccade (time only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return accuracy_dist

### Accuracy between secondary saccade (time and AOI constraint) and stimuli: 

In [43]:
def accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
    for key,value in stimuli_location.items(): 
        stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [MCS norm]'].iloc[0]
            y1 = row1['Gaze point Y [MCS norm]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance

    print("Distance between secondary saccade (time + AOI only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return accuracy_dist

### Corrective saccades between primary saccade and secondary saccade (time + AOI) 

In [44]:
def correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index[0]
        index_secondary = df_reset[df_reset['index'] == value].index[0]
        correctiveSaccade[key] = int(index_secondary) - int(index_primary)

    print("Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return correctiveSaccade

### Corrective saccades between primary saccade and secondary saccade (time) 

In [45]:
def correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTime.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index
        if len(index_primary) != 0:
            index_secondary = df_reset[df_reset['index'] == value].index[0]

            correctiveSaccade[key] = int(index_secondary) - int(index_primary[0])

    print("Number of corrective saccade between secondary saccade (time only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return correctiveSaccade

## MAIN LOOP

In [ ]:
import os
import glob

times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

folder_path = 'plusLenses'
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
for file_path in excel_files:
    print(file_path)
#     file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTime)
    amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    saccade_latency = saccadeLatency(primarySaccadeTime, times_dict)
    accuracy_dist = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
    accuracy_dist = accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade)
    accuracy_dist = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    correctiveSaccade = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    correctiveSaccade = correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade)
    
    print(" ")


In [50]:
times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

#     file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
dataExport = pd.read_excel('plusLenses/amblypiaStudySanaRight4.xlsx')
dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)

primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTime)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
saccade_latency = saccadeLatency(primarySaccadeTime, times_dict)
accuracy_dist = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
correctiveSaccade = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
correctiveSaccade = correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade)



,Eyetracker timestamp [μs],Event,Event value,Gaze point X [MCS norm],Gaze point Y [MCS norm],...,AOI hit [catStimulus-final (1) - cat11],Gaze point left X [MCS norm],Gaze point left Y [MCS norm],Gaze point right X [MCS norm],Gaze point right Y [MCS norm]
6934,NaN,Eye tracker Calibration end,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
6935,NaN,VideoStimulusStart,slothStimulus-final (1),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
6936,0.0,NaN,NaN,8.584086,7.275423,...,NaN,8.584086,7.275423,NaN,NaN
6937,8334.0,NaN,NaN,8.200998,8.418319,...,NaN,8.200998,8.418319,NaN,NaN
6938,16667.0,NaN,NaN,8.163522,9.328186,...,NaN,8.163522,9.328186,NaN,NaN


,Eyetracker timestamp [μs],Event,Event value,Gaze point X [MCS norm],Gaze point Y [MCS norm],...,AOI hit [catStimulus-final (1) - cat11],Gaze point left X [MCS norm],Gaze point left Y [MCS norm],Gaze point right X [MCS norm],Gaze point right Y [MCS norm]
14129,27091691.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
14130,27100025.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
14131,27108358.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
14132,27116691.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
14133,NaN,RecordingEnd,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


6934   NaN
6935   NaN
6936   NaN
6937   NaN
6938   NaN
6939   NaN
6940   NaN
6941   NaN
6943   NaN
6953   NaN
6956   NaN
6961   NaN
6970   NaN
6971   NaN
6974   NaN
6975   NaN
6976   NaN
6977   NaN
6978   NaN
6979   NaN
6980   NaN
6981   NaN
6982   NaN
6983   NaN
6984   NaN
6985   NaN
6986   NaN
6987   NaN
6988   NaN
6989   NaN
6990   NaN
6991   NaN
6992   NaN
6993   NaN
6994   NaN
6995   NaN
6996   NaN
6997   NaN
6998   NaN
6999   NaN
7000   NaN
7001   NaN
7002   NaN
7003   NaN
7004   NaN
7005   NaN
7006   NaN
7007   NaN
7008   NaN
7009   NaN
7010   NaN
7011   NaN
7012   NaN
7013   NaN
7014   NaN
7015   NaN
7016   NaN
7017   NaN
7018   NaN
7019   NaN
7020   NaN
7021   NaN
7022   NaN
7023   NaN
7024   NaN
7025   NaN
7026   NaN
7027   NaN
7028   NaN
7029   NaN
7030   NaN
7031   NaN
7032   NaN
7033   NaN
7034   NaN
7035   NaN
7036   NaN
7037   NaN
7038   NaN
7039   NaN
7040   NaN
7041   NaN
7042   NaN
7043   NaN
7044   NaN
7045   NaN
7046   NaN
7047   NaN
7048   NaN
7049   NaN
7050   NaN

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7032,
             'AOI hit [slothStimulus-final (1) - sloth2]': 7421,
             'AOI hit [slothStimulus-final (1) - sloth3]': 7712,
             'AOI hit [slothStimulus-final (1) - sloth4]': 8120,
             'AOI hit [slothStimulus-final (1) - sloth5]': 8244,
             'AOI hit [slothStimulus-final (1) - sloth6]': 8744,
             'AOI hit [slothStimulus-final (1) - sloth8]': 9350,
             'AOI hit [catStimulus-final (1) - cat1]': 11161,
             'AOI hit [catStimulus-final (1) - cat2]': 11111,
             'AOI hit [catStimulus-final (1) - cat5]': 11838,
             'AOI hit [catStimulus-final (1) - cat6]': 12489,
             'AOI hit [catStimulus-final (1) - cat7]': 12747,
             'AOI hit [catStimulus-final (1) - cat9]': 13338,
             'AOI hit [catStimulus-final (1) - cat10]': 13728,
             'AOI hit [catStimulus-final (1) - cat11]': 14051})

AOI hit [slothStimulus-final (1) - sloth1] 6979
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth2] 7320
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth3] 7585
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth4] 7889
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth5] 8201
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth6] 8520
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth7] 8790
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth8] 9095
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth9] 9401
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth10] 9723
nan nan nan nan
AOI hit [slothStimulus-final (1) - sloth11] 9987
nan nan nan nan
AOI hit [catStimulus-final (1) - cat1] 11111
nan nan nan nan
AOI hit [catStimulus-final (1) - cat3] 11548
3.2812319999999997 4.821007000000001 nan nan
AOI hit [catStimulus-final (1) - cat4] 11838
7.1287408434139055 -2.5229135794414916 2.542122 5.106731
AOI hit [catStimulus-final (1) - cat6] 12

IndexError: index 0 is out of bounds for axis 0 with size 0

### Extracting relevant features from data metric

### TODO: Identify number of saccades in each AOI manually and where they are

In [ ]:
saccade_metrics_location = defaultdict(int)
saccade_metrics_number = defaultdict(int)

columns_to_keep = ['Gaze point X (MCSnorm)', 'Gaze point Y (MCSnorm)', 'Gaze point left X (MCSnorm)', 
                          'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)', 'Gaze point right Y (MCSnorm)']
for i in times_dict.keys():
    all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]

    total_saccades = 0
    df_saccades = pd.DataFrame()
    for index, row in all_possible_hits.iterrows(): 
        time = row['Eyetracker timestamp']
        #adding latency
        if (time/1000000 > (times_dict[i])+0.2) and (time/1000000 < (times_dict[i] + 2.5)):
            df_saccades = df_saccades.append(row[columns_to_keep], ignore_index=True)
        
            total_saccades +=1
    saccade_metrics_number[i] = total_saccades
    saccade_metrics_location[i] = df_saccades
  

saccade_metrics_location
saccade_metrics_number


In [ ]:
# dataMetricSaccades = dataMetric.iloc[:,list(range(520, dataMetric.shape[1]))]


In [ ]:
# dataMetricSaccades[['Number_of_saccades_in_AOI.bottomLeftCat1',
#        'Number_of_saccades_in_AOI.bottomLeftSloth',
#        'Number_of_saccades_in_AOI.bottomRightCat',
#        'Number_of_saccades_in_AOI.bottomRightCat2',
#        'Number_of_saccades_in_AOI.bottomRightSloth',
#        'Number_of_saccades_in_AOI.centerCat',
#        'Number_of_saccades_in_AOI.centerCat2',
#        'Number_of_saccades_in_AOI.centerSloth',
#        'Number_of_saccades_in_AOI.topCenterCat',
#        'Number_of_saccades_in_AOI.topCenterSloth',
#        'Number_of_saccades_in_AOI.topLeftCat',
#        'Number_of_saccades_in_AOI.topRightCat',
#        'Number_of_saccades_in_AOI.topRightSloth',
#        'Number_of_saccades_in_AOI.topRightSloth1',
#        'Time_to_entry_saccade.bottomLeftCat1']]

In [ ]:
# dataMetricSaccades['Time_to_entry_saccade.centerCat2']